In [1]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from multiprocessing import Pool

In [2]:
def read_large_file(file_path):
    """파일에서 텍스트를 읽어 리스트로 반환."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

In [3]:
def calculate_similarity(chunk):
    """주어진 텍스트 조각에 대해 유사도를 계산."""
    indices, texts = chunk
    vectorizer = TfidfVectorizer().fit_transform(texts)
    similarity_matrix = cosine_similarity(vectorizer)
    results = []
    for i in range(len(texts)):
        for j in range(i + 1, len(texts)):
            if similarity_matrix[i, j] > 0.8:  # 유사도가 0.8 이상인 경우
                results.append((indices[i], indices[j], similarity_matrix[i, j]))
    return results

In [4]:
def process_in_chunks(file_path, chunk_size=1000):
    """파일을 청크 단위로 처리."""
    texts = read_large_file(file_path)
    num_texts = len(texts)
    chunks = [
        (list(range(i, min(i + chunk_size, num_texts))),
         texts[i:min(i + chunk_size, num_texts)])
        for i in range(0, num_texts, chunk_size)
    ]
    return chunks

In [5]:
def save_results(results, output_file):
    """결과를 파일에 저장."""
    with open(output_file, 'w', encoding='utf-8') as file:
        for idx1, idx2, score in results:
            file.write(f"{idx1}\t{idx2}\t{score:.2f}\n")

In [ ]:
if __name__ == "__main__":
    input_file = '복무Q&A.txt'  # 입력 텍스트 파일
    output_file = 'similar_sentences.txt'  # 출력 파일
    chunk_size = 1000  # 청크 크기 설정

    chunks = process_in_chunks(input_file, chunk_size)

    # 멀티프로세싱 사용
    with Pool() as pool:
        all_results = pool.map(calculate_similarity, chunks)

    # 결과 병합
    flattened_results = [item for sublist in all_results for item in sublist]

    # 파일에 저장
    save_results(flattened_results, output_file)

    print(f"유사도 계산 완료. 결과가 {output_file}에 저장되었습니다.")